In [1]:
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0
!pip install accelerate peft bitsandbytes transformers trl evaluate sentencepiece wandb

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found

In [2]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training, PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
# load model (only change this)
model_name = "mob2711/phi-3-vi-sft-1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
tokenizer = AutoTokenizer.from_pretrained("mob2711/phi-3-vi-sft-1", trust_remote_code=True)
config = PeftConfig.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True, device_map="auto")
model = PeftModel.from_pretrained(model, model_name)

cuda


tokenizer_config.json:   0%|          | 0.00/3.27k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/432M [00:00<?, ?B/s]

In [4]:
# you have to access https://huggingface.co/datasets/story_cloze, follow up on the tutorial, and download the data

import json

data = []

with open("data\hellaswag.jsonl", 'r') as file:
    for line in file:
        data.append(json.loads(line))

test = pd.DataFrame(data)
test

# preprocessing data


,ind,id,activity_label,ctx_a,ctx_b,ctx,dataset,split_type,ending_options
0,0,v_-0NfmccuL9Q,Changing car wheel,A man has a few tools and is pumping his car u...,he,A man has a few tools and is pumping his car u...,activitynet,zeroshot,[uses the tool to take off all of the nuts one...
1,1,v_-0NfmccuL9Q,Changing car wheel,A man has a few tools and is pumping his car u...,then he,A man has a few tools and is pumping his car u...,activitynet,zeroshot,[puts in a new tire and takes the lug nuts off...
2,2,v_-0NfmccuL9Q,Changing car wheel,He uses the tool to take off all of the nuts o...,he,He uses the tool to take off all of the nuts o...,activitynet,zeroshot,"[then pulls up one of the spare tire, balances..."
3,5,v_-2DajzzRwn0,Tug of war,Several teams of people are seen playing a gam...,the people,Several teams of people are seen playing a gam...,activitynet,zeroshot,[pull more and more objects along the front wh...
4,6,v_-2VNLsk1s34,Shot put,A large man is seen stepping up into a circle ...,several more men,A large man is seen stepping up into a circle ...,activitynet,indomain,[jump into the circle and put end up in goals....
...,...,...,...,...,...,...,...,...,...
10045,50636,83326,Relationships,[header] How to practice good elevator etiquet...,,[header] How to practice good elevator etiquet...,wikihow,indomain,[[title] Stand back when the elevator doors cl...
10046,50644,83339,Home and Garden,[header] How to dry water under wood floor [ti...,,[header] How to dry water under wood floor [ti...,wikihow,indomain,"[If the floor is dry, run your fingers over th..."
10047,50645,83342,Food and Entertaining,[header] How to soften honey [title] Boil a ke...,,[header] How to soften honey [title] Boil a ke...,wikihow,indomain,[[title] Pour the boiled water into the bowl. ...
10048,50671,83385,Hobbies and Crafts,[header] How to put makeup on your photo insta...,,[header] How to put makeup on your photo insta...,wikihow,zeroshot,"[On the search bar, type in "" perfect365 "" and..."


In [5]:
print(test.iloc[10046]['ctx'])
print(test.iloc[10046]['ending_options'])

[header] How to dry water under wood floor [title] Remove all floor coverings. [step] The area underneath your wood floor won't dry while the wood is still wet. If your wood flooring is covered by wet rugs, mats, or carpeting, you'll need to remove them immediately.
['If the floor is dry, run your fingers over the area to collect any damp dirt or soil to wash it. [substeps] If the floor is not truly dry, you can let it sit overnight and then use a spray bottle to dry.', 'Immediately remove any rugs and any mats that they might be covering up in the drying process. [title] Remove sanding from your wood floor.', 'If your carpets and rugs have been soaked and they are not immediately cleaned, they will usually need to be discarded due to mildew concerns. [substeps] A professional carpet cleaning company may be able to help save your carpet and treat it for mold.', "Your wood floor may not be able to stay wet for a while, and excess moisture can lead to mold and mildew. [substeps] Before a

In [6]:
choice = ['A', 'B', 'C', 'D', 'E', 'F']

def eval(df, idx):
    doc = df.iloc[idx]
    text_choice = ''
    for i in range(len(doc['ending_options'])):
        text_choice += '\n' + choice[i] + '. ' + doc['ending_options'][i]
    prompt = f"<|user|>\nThis is a commonsense multiple choice question (with answers) about {doc['activity_label'].lower()}. Choose one answer to continue the following paragraph:\n" \
                + "Paragraph: " + doc['ctx'] \
                + "\n" \
                + text_choice \
                + "\n" \
                + "<|assistant|>\n" \
                + "Answer: " 
#     return prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    with torch.no_grad():
            outputs = model(input_ids=input_ids)
    logits = outputs.logits  # shape (batch_size, sequence_length, vocab_size)
    next_token_logits = logits[:, -1, :]  # shape (batch_size, vocab_size)
#         print(logits)

    next_token_logits = next_token_logits.flatten()
    next_token_probs = torch.nn.functional.softmax(next_token_logits, dim=-1).cpu()
    tokens_of_interest = [
        tokenizer("A", add_special_tokens=False).input_ids[-1],
        tokenizer("B", add_special_tokens=False).input_ids[-1],
        tokenizer("C", add_special_tokens=False).input_ids[-1],
        tokenizer("D", add_special_tokens=False).input_ids[-1],
#         tokenizer("E", add_special_tokens=False).input_ids[-1],
    ]
    probs = next_token_probs[tokens_of_interest].tolist()
    pred = {0: "A", 1: "B", 2: "C", 3: "D"}[np.argmax(probs)]
    return pred

print(eval(test, 0))

2024-05-09 10:22:25.544031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 10:22:25.544143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 10:22:25.657823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


A


In [7]:
from tqdm import tqdm

num = {"A": 0, "B": 1, "C": 2, "D": 3}

ans = []
for i in tqdm(range(len(test))):
    ans.append(num[eval(test, i)])

100%|██████████| 10050/10050 [1:48:30<00:00,  1.54it/s]


In [8]:
test['answer'] = ans
output_path = ""
test[['answer']].to_csv(output_path, index=False)